In [1]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
import numpy as np
from coypu.utils.functions import json_to_csv
from json import loads, load
import os

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

coypu.utils


In [2]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
    
fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

In [ ]:
print(query_3_fdq_desc)
print(query_3_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_3_fdq_ex))

# fdquery.post_query(query_3_fdq_ex)
# fdquery.to_save('../../query_results/', filename='query_3_fdq_ex')

In [3]:
from functools import reduce

a = reduce(lambda a,b : a+b, [1,2,3,45,])
print(a)

def load_json(filename):
    with open(filename) as js:
        json_data = load(js)
        return json_data
    
def merge_jsons_to_csv(filepath, save_path, outfile, columns):
    df = reduce(lambda df1, df2: pd.concat([df1, df2]), \
                [json_to_csv(load_json(filepath+filename)['results']['bindings'], save_path, filename, columns, True) for filename in os.listdir(filepath)])
    
    df.to_csv(save_path+outfile, encoding='utf-8', index=False)
    
    
columns=[var+'.value' for var in load_json('../../query_results/query_3/query3_uk.json')['head']['vars']]
print(columns)
merge_jsons_to_csv('../../query_results/query_3/', '../../query_results/', 'query_3_fdq_ex.csv', columns)

51
['year.value', 'value.value', 'population.value']


In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_3_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_3_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port))

df = pd.read_sql(query_3_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_3_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('../../query_results/query_3_fdq_ex_sql.csv', encoding='utf-8')

df['WikiData-WB(AvgLifeExpectancy)'] = np.abs(df['year_avg_exp_wiki']-df['year_avg_exp_WB'])
df.head(10)

In [ ]:
import plotly.express as px
fig = px.choropleth(df[df['year_WB']==2016], locations="country_code",
                    color="WikiData-WB(AvgLifeExpectancy)", # lifeExp is a column of gapminder
                    hover_name="country_name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.write_html('../../query_results/query_3_fdq_ex_sql.html')
fig.show()